##  In this notebook, I have coded a ridersharing program. 

### A road is the two-dimensional cartesian product.   We will genate  a car with some features. The car will take the main rider in and drive in the direction of the rider's destination. Then, other riders are randomly generated on the map, each with a location and a direction. Given the destination of the main rider, the car will pick up riders on the way, if there is enough space in the car. This program ends when the main rider is droppe at her destination.


### Define the rider class. A rider has a name, a location, a destination and a number of seats requested.

In [1]:
class Rider():
    
    '''
    A rider is defined by its location and the number of seats requested and a destination.
    Location can be an integer or float greater or equal to 0.
    The road starts at 0 and has a size of 100.
    '''
        
    def __init__(self, name, loc, des, sea):
        self.name = name # identity of the rider
        self.loc = loc # location is a two-dimensional list
        self.sea = sea # how many seats do you need
        self.des = des
        self.dist = dist
    
   

## Define a car class. 

### A car is defined by its speed (Slow or Fast) and a number of available seats.

In [2]:
class Car():
    
    def __init__(self, speed, sea):
        self.speed = speed #fast or slow
        self.sea = sea # 1, 2 or 3
        

## Define a random car generator. 

### This function generate a car. Speed is randomly assigned, and a number of seats is randomly assigned as a number between 2 and "maxspace".

In [3]:
def car_generator(maxspace):
    import random
    speed = random.randint(1,2)
    if speed == 1:
        speed = 'fast'
    else:
        speed = 'slow'
    
    space = random.randint(2,maxspace)
    
    print(f'Your car is {speed.upper()} and has a total of {space} seat(s).')
    return Car(speed, space)
    

## Define a random rider generator.

### A rider is randomly generated. The input to that function is the name of the rider. Location, destination and a number of seats requested are randomly assigned.

In [4]:
def rider_generator(name):
    import random
    '''
    generate a random location (2x2 list of floats) between -100 and 100
    generate a random number (integer) of seats requested between 1 and 3
    generate a random destination (2x2 list of floats) between -100 and 100
    '''
    loca = [round(random.uniform(-100,100),2),round(random.uniform(-100,100),2)]
    seats = random.randint(1,2)
    dest = [round(random.uniform(-100,100),2),round(random.uniform(-100,100),2)]
    return Rider(name,loca, dest, seats)
    

## Define a map of other riders.

### This function will generate n riders and name the ith rider R_i. The main rider will be named R_0.

In [5]:
def riders_map(n):
    '''
    The input is n, the number of riders you want to generate.
    '''
    import random
    riderList = []
    for num in range(1,n +1):
        riderList.append(rider_generator(f'R_{num}'))
    
    return riderList

## Define a function that measures distance between two points on the 2-dimensional cartesian product.

### We will need that function when we identify the riders "on the way". The car needs to know where are the closest riders who ride in the direction of the main rider destination.

In [6]:
def dist(a,b):
    
    dst = 0
    
    for n in range(0,min(len(a),len(b))):
    
        dst += (a[n]-b[n])**2 
    
    return dst**(1/2)

## Define a function that locates all other riders that are "on the way". 

### This function returns the riders to pick up who are the closest from some_rider ordered by distance.


In [7]:
def pickups_locator(ridersmap,some_rider):
    '''
    This function takes as input the list of generated riders and some_rider.
    
    condition_1 : is the rider between some_rider location and destination?
    condition_2 : is the rider location closer to some_rider destination than some_rider location is?
    condition_3 : is the rider destination closer to some_rider destination that some_rider location is?
    
    '''
    riders_on_way = []
    
    for rider in ridersmap:
        
        condition_1 = dist(rider.loc, some_rider.loc) < dist(some_rider.loc, some_rider.des)
        condition_2 = dist(rider.loc, some_rider.des) < dist(some_rider.loc, some_rider.des)
        condition_3 = dist(rider.des, some_rider.des) < dist(some_rider.loc, some_rider.des)
        
        if  condition_1 and condition_2 and condition_3:
            rider.dist = round(dist(rider.loc, some_rider.loc),2)
            riders_on_way.append(rider)
    
    import operator
    riders_on_way.sort(key=operator.attrgetter('dist'))
            
    return riders_on_way
            

### Among the riders on the way, if there is enough space in the car, pick up the nearest rider. Then, determine who should be dropped off first. With respect to the location of the last pick-up and the destination of the first drop-off, locate riders  who are on the way.

## Define a function that runs the ride, find pick ups, makes drop offs until the main rider is dropped off at its destination.

determine a drop off based on the last pick-up and who is in the car.

In [8]:
def dropoff_dist(inthecar, location):
    for rider in inthecar:
        rider.dist = dist(location, rider.loc)
    
    import operator
    inthecar.sort(key=operator.attrgetter('dist'))

In [9]:
def ride(mainrider, thecar, ridersontheway):

    inthecar = []

    location = mainrider.loc
    inthecar.append(mainrider)
    distance_ran = 0
    thecar.sea -= 1
    dropoff_dist(inthecar, location)
    
    

    while mainrider in inthecar and ridersontheway != []:

        
        if dist(location, inthecar[0].des) < dist(location, ridersontheway[0].loc):
            
            distance_ran += dist(location, inthecar[0].des)
            location = inthecar[0].des
            thecar.sea += inthecar[0].sea        
            print(f'Rider {inthecar[0].name} has been dropped off at {inthecar[0].des}')
            inthecar.remove(inthecar[0])
            ridersontheway.remove(ridersontheway[0])


        elif dist(location, inthecar[0].des) > dist(location, ridersontheway[0].loc) and thecar.sea >= ridersontheway[0].sea:

            inthecar.append(ridersontheway[0])
            print(f'Rider {ridersontheway[0].name} has been picked up at {ridersontheway[0].loc}')
            distance_ran += dist(location, ridersontheway[0].loc )
            location =  ridersontheway[0].loc
            thecar.sea -= ridersontheway[0].sea
            ridersontheway.remove(ridersontheway[0])
            dropoff_dist(inthecar, location)

        else: 
            ridersontheway.remove(ridersontheway[0])
        
    while mainrider in inthecar:
        
        for rider in inthecar:
            
            distance_ran += dist(location, inthecar[0].des)
            location = inthecar[0].des
            thecar.sea += inthecar[0].sea        
            print(f'Rider {inthecar[0].name} has been dropped off at {inthecar[0].des}')
            inthecar.remove(inthecar[0])
    
    print(f'The minimum distance was: {round(dist(mainrider.loc, mainrider.des),2)}\n the total distance driven was :{round(distance_ran,2)}')

        
    
    


## Let's run the program. 

### Our main rider is named r_0. We are generating a map with 500 riders. Given the features of the main rider, we generate the list of riders on the way. Also, we are generating a car at random.

In [14]:
main_rider = rider_generator('R_0')
print(f"R_0's location is {main_rider.loc} and her destination is {main_rider.des}")
test_map = riders_map(500)
car_test = car_generator(10)
ridersontheway = pickups_locator(test_map, main_rider)



R_0's location is [-94.37, 37.41] and her destination is [26.1, -24.14]
Your car is SLOW and has a total of 6 seat(s).


In [15]:
ride(main_rider, car_test, ridersontheway)

Rider R_293 has been picked up at [-87.27, 34.07]
Rider R_135 has been picked up at [-83.77, 37.92]
Rider R_35 has been picked up at [-79.71, 33.85]
Rider R_104 has been picked up at [-70.96, 39.13]
Rider R_104 has been dropped off at [-38.7, 76.7]
Rider R_49 has been picked up at [-50.88, 29.2]
Rider R_49 has been dropped off at [37.35, 59.7]
Rider R_35 has been dropped off at [11.36, -1.47]
Rider R_436 has been picked up at [-48.2, -48.01]
Rider R_436 has been dropped off at [-18.13, -74.06]
Rider R_352 has been picked up at [3.16, 33.52]
Rider R_352 has been dropped off at [74.02, 17.31]
Rider R_251 has been picked up at [-1.38, 6.98]
Rider R_332 has been picked up at [3.66, 44.75]
Rider R_332 has been dropped off at [7.4, 32.5]
Rider R_279 has been picked up at [2.03, 8.68]
Rider R_279 has been dropped off at [-21.19, 43.4]
Rider R_460 has been picked up at [7.99, 26.62]
Rider R_460 has been dropped off at [-0.11, -34.22]
Rider R_251 has been dropped off at [40.91, -63.37]
Rider R_

## Future extension of the code: display the whole map (black dot for each rider) 
                display the map of riders on the way (green dot for each rider on the way)
                
                extra: display their location and destination and an edge between them
                